# <center>Curso de Modelagem de Dados para IA - PARTE 11</center>

<img src="image.jpg" alt="Drawing" style="width: 300px;"/>


## Operações Groupby sobre os dados
### Agregando dados: split-apply-combine e groupby
Às vezes, o processamento de dados que queremos fazer envolve manipulações mais complicadas dos dados brutos, além de simplesmente adicionar novas colunas às tabelas de dados existentes ou realizar operações agregadas em um conjunto de dados inteiro. Como uma tabela de dados potencialmente envolve muita estrutura interna associada aos valores das entradas na tabela, às vezes queremos agregar subconjuntos dos dados para calcular algumas propriedades sobre esses subconjuntos.

As operações *Groupby* representam um conjunto extremamente poderoso de recursos em pandas para manipular dataframes, fornecendo suporte para uma classe geral das chamadas operações "split-apply-combine". Isso significa, por exemplo, que podemos:

- Dividir um dataframe em grupos com base na identidade de uma chave específica ou algum outro critério
- Aplicar uma função de agregação em cada um dos subgrupos
- Combinar as informações agregadas de volta em um único dataframe

Por exemplo, cada linha no dataframe de rebatidas contém informações sobre um único jogador em um único ano. E se quiséssemos saber os totais de todas essas estatísticas, ano a ano? Podemos criar um novo dataframe de maneira muito simples agrupando pelo '<span style="font-family: 'Courier'">yearID</span>' e somando cada um desses grupos ano a ano:

- dividir o dataframe em um grupo para cada ano
- aplicar a função de agregação de soma (soma) dentro de cada grupo
- combinar os dados somados ano a ano em um único dataframe

In [1]:
import glob, os
import pandas as pd

In [2]:
def read_all_databank_core_csv(directory):
    """
    read all csv files in the specified baseball databank directory and
    populate a dictionary storing each of the tables keyed to its name
    """
    dfs = {}
    files = glob.glob('{}/*.csv'.format(directory))
    for f in files:
        d, name = os.path.split(f)
        table = os.path.splitext(name)[0]
        df = pd.read_csv(f)
        dfs[table] = df
    return dfs

bbdfs = read_all_databank_core_csv('data/baseballdatabank/core')

In [3]:
batting = bbdfs['Batting']
pitching = bbdfs['Pitching']
teams = bbdfs['Teams']

In [7]:
batting['1B'] = batting['H'] - batting['2B'] - batting['3B'] - batting['HR']
teams['1B'] = teams['H'] - teams['2B'] - teams['3B'] - teams['HR']

In [8]:
batting_by_year = batting.groupby('yearID').sum().reset_index()
batting_by_year.head()

,yearID,stint,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,1B
0,1871,115,2296,10822,2659,3101,434,239,47,1783.0,441.0,123.0,393,175.0,0.0,0.0,0.0,0.0,74.0,2381
1,1872,172,3305,15663,3390,4467,581,145,37,2132.0,269.0,134.0,263,264.0,0.0,0.0,0.0,0.0,97.0,3704
2,1873,128,3604,16959,3580,4926,570,211,47,2331.0,314.0,131.0,335,278.0,0.0,0.0,0.0,0.0,122.0,4098
3,1874,126,4199,19104,3470,5224,634,194,40,2252.0,242.0,97.0,238,357.0,0.0,0.0,0.0,0.0,107.0,4356
4,1875,248,6248,26833,4234,6812,839,273,40,2710.0,629.0,320.0,249,675.0,0.0,0.0,0.0,0.0,142.0,5660


Como alternativa, em vez de realizar agrupamento ano a ano para obter estatísticas anuais de toda a liga, poderíamos agrupar por jogador (<span style="font-family: 'Courier'">playerID</span>) para obter estatísticas de rebatidas da carreira para cada jogador:

In [9]:
pl_bat = batting.groupby('playerID').sum().reset_index()
pl_bat.head()

,playerID,yearID,stint,G,AB,R,H,2B,3B,HR,...,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,1B
0,aardsda01,18084,9,331,4,0,0,0,0,0,...,0.0,0.0,0,2.0,0.0,0.0,1.0,0.0,0.0,0
1,aaronha01,45195,23,3298,12364,2174,3771,624,98,755,...,240.0,73.0,1402,1383.0,293.0,32.0,21.0,121.0,328.0,2294
2,aaronto01,13768,7,437,944,102,216,42,6,13,...,9.0,8.0,86,145.0,3.0,0.0,9.0,6.0,36.0,155
3,aasedo01,25786,13,448,5,0,0,0,0,0,...,0.0,0.0,0,3.0,0.0,0.0,0.0,0.0,0.0,0
4,abadan01,6010,3,15,21,1,2,0,0,0,...,0.0,1.0,4,5.0,0.0,0.0,0.0,0.0,1.0,2


Como um aparte técnico, em ambos os exemplos acima, encadeamos várias operações em uma expressão, concluindo cada uma com o método <span style="font-family: 'Courier'">reset_index()</span>. Para que serve essa operação? Lembre-se de que o índice de um dataframe é o grupo de rótulos para cada linha. Nos dataframes brutos que criamos quando lemos os arquivos csv, o índice era simplesmente um conjunto crescente de inteiros sobre cada uma das linhas. Quando executamos uma operação groupby como as acima, um novo dataframe é retornado e o índice desse dataframe é o conjunto de rótulos que agrupamos, por exemplo, o <span style="font-family: 'Courier'">yearID</span> no primeiro exemplo e o <span style="font-family: 'Courier'">playerID</span> no segundo exemplo. Às vezes é útil manter a chave groupby como o índice, mas às vezes você pode querer colocar a chave de volta como uma coluna regular e usar o incremento de inteiros como um índice. É isso que o método <span style="font-family: 'Courier'">reset_index()</span> faz. Sinta-se à vontade para experimentar isso no notebook, por exemplo, removendo a chamada para <span style="font-family: 'Courier'">reset_index</span> para que você possa inspecionar o dataframe que é retornado sem essa chamada.

Observe que as estatísticas "Slash Line" que adicionamos ao dataframe de rebatidas no exercício anterior não foram agregadas corretamente aqui no dataframe pl_bat. Isso ocorre porque essas estatísticas são todas médias de contagens ano a ano. A média de rebatidas na carreira de um jogador, no entanto, não é a soma de suas médias de rebatidas ano a ano (ou mesmo a média dessas médias). Em vez disso, as estatísticas Slash Line de toda a carreira precisariam ser recalculadas com base nas contagens agregadas no dataframe pl_bat, como no código abaixo. Assim, embora um dataframe seja facilmente extensível, também precisamos ter em mente se a adição de novos tipos de dados altera a semântica da tabela.

In [10]:
pl_bat['BA'] = pl_bat['H'] / pl_bat['AB']
pl_bat['OBP'] = (pl_bat['H']+pl_bat['BB']+pl_bat['HBP']) / (pl_bat['AB']+pl_bat['BB']+pl_bat['HBP']+pl_bat['SF'])
pl_bat['SLG'] = (pl_bat['1B']+2*pl_bat['2B']+3*pl_bat['3B']+4*pl_bat['HR']) / pl_bat['AB']
pl_bat['OPS'] = (pl_bat['OBP']+pl_bat['SLG'])

In [11]:
pl_bat.head()

,playerID,yearID,stint,G,AB,R,H,2B,3B,HR,...,IBB,HBP,SH,SF,GIDP,1B,BA,OBP,SLG,OPS
0,aardsda01,18084,9,331,4,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000
1,aaronha01,45195,23,3298,12364,2174,3771,624,98,755,...,293.0,32.0,21.0,121.0,328.0,2294,0.304998,0.373949,0.554513,0.928462
2,aaronto01,13768,7,437,944,102,216,42,6,13,...,3.0,0.0,9.0,6.0,36.0,155,0.228814,0.291506,0.327331,0.618836
3,aasedo01,25786,13,448,5,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0,0.000000,0.000000,0.000000,0.000000
4,abadan01,6010,3,15,21,1,2,0,0,0,...,0.0,0.0,0.0,0.0,1.0,2,0.095238,0.240000,0.095238,0.335238
